<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# DKN : Deep Knowledge-Aware Network for News Recommendation

DKN \[1\] is a deep learning model which incorporates information from knowledge graph for better news recommendation. Specifically, DKN uses TransX \[2\] method for knowledge graph representation learning, then applies a CNN framework, named KCNN, to combine entity embedding with word embedding and generate a final embedding vector for a news article. CTR prediction is made via an attention-based neural scorer. 

## Properties of DKN:

- DKN is a content-based deep model for CTR prediction rather than traditional ID-based collaborative filtering. 
- It makes use of knowledge entities and common sense in news content via joint learning from semantic-level and knowledge-level representations of news articles.
- DKN uses an attention module to dynamically calculate a user's aggregated historical representaition.


## Data format:

### DKN takes several files as input as follows:

- **training / validation / test files**: each line in these files represents one instance. Impressionid is used to evaluate performance within an impression session, so it is only used when evaluating, you can set it to 0 for training data. The format is : <br> 
`[label] [userid] [CandidateNews]%[impressionid] `<br> 
e.g., `1 train_U1 N1%0` <br> 

- **user history file**: each line in this file represents a users' click history. You need to set `history_size` parameter in the config file, which is the max number of user's click history we use. We will automatically keep the last `history_size` number of user click history, if user's click history is more than `history_size`, and we will automatically pad with 0 if user's click history is less than `history_size`. the format is : <br> 
`[Userid] [newsid1,newsid2...]`<br>
e.g., `train_U1 N1,N2` <br> 

- **document feature file**: It contains the word and entity features for news articles. News articles are represented by aligned title words and title entities. To take a quick example, a news title may be: <i>"Trump to deliver State of the Union address next week"</i>, then the title words value may be `CandidateNews:34,45,334,23,12,987,3456,111,456,432` and the title entitie value may be: `entity:45,0,0,0,0,0,0,0,0,0`. Only the first value of entity vector is non-zero due to the word "Trump". The title value and entity value is hashed from 1 to `n` (where `n` is the number of distinct words or entities). Each feature length should be fixed at k (`doc_size` parameter), if the number of words in document is more than k, you should truncate the document to k words, and if the number of words in document is less than k, you should pad 0 to the end. 
the format is like: <br> 
`[Newsid] [w1,w2,w3...wk] [e1,e2,e3...ek]`

- **word embedding/entity embedding/ context embedding files**: These are `*.npy` files of pretrained embeddings. After loading, each file is a `[n+1,k]` two-dimensional matrix, n is the number of words(or entities) of their hash dictionary, k is dimension of the embedding, note that we keep embedding 0 for zero padding. 

In this experiment, we used GloVe\[4\] vectors to initialize the word embedding. We trained entity embedding using TransE\[2\] on knowledge graph and context embedding is the average of the entity's neighbors in the knowledge graph.<br>

## MIND dataset

MIND dataset\[3\] is a large-scale English news dataset. It was collected from anonymized behavior logs of Microsoft News website. MIND contains 1,000,000 users, 161,013 news articles and 15,777,377 impression logs. Every news article contains rich textual content including title, abstract, body, category and entities. Each impression log contains the click events, non-clicked events and historical news click behaviors of this user before this impression.

In this notebook we are going to use a subset of MIND dataset, **MIND demo**. MIND demo contains 500 users, 9,432 news articles  and 6,134 impression logs. 

For this quick start notebook, we are providing directly all the necessary word embeddings, entity embeddings and context embedding files.

## Global settings and imports

In [1]:
import sys
sys.path.append("../../")

import os
from tempfile import TemporaryDirectory
import papermill as pm
import scrapbook as sb
import tensorflow as tf

from reco_utils.recommender.deeprec.deeprec_utils import download_deeprec_resources, prepare_hparams
from reco_utils.recommender.deeprec.models.dkn import DKN
from reco_utils.recommender.deeprec.io.dkn_iterator import DKNTextIterator

print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0]
Tensorflow version: 1.15.2


## Download and load data

In [2]:
tmpdir = TemporaryDirectory()
data_path = os.path.join(tmpdir.name, "mind-demo-dkn")

yaml_file = os.path.join(data_path, r'dkn.yaml')
train_file = os.path.join(data_path, r'train_mind_demo.txt')
valid_file = os.path.join(data_path, r'valid_mind_demo.txt')
test_file = os.path.join(data_path, r'test_mind_demo.txt')
news_feature_file = os.path.join(data_path, r'doc_feature.txt')
user_history_file = os.path.join(data_path, r'user_history.txt')
wordEmb_file = os.path.join(data_path, r'word_embeddings_100.npy')
entityEmb_file = os.path.join(data_path, r'TransE_entity2vec_100.npy')
contextEmb_file = os.path.join(data_path, r'TransE_context2vec_100.npy')
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/deeprec/', tmpdir.name, 'mind-demo-dkn.zip')
    

100%|██████████| 11.3k/11.3k [00:01<00:00, 8.60kKB/s]


## Create hyper-parameters

In [3]:
epochs = 10
history_size = 50
batch_size = 100

In [4]:
hparams = prepare_hparams(yaml_file,
                          news_feature_file = news_feature_file,
                          user_history_file = user_history_file,
                          wordEmb_file=wordEmb_file,
                          entityEmb_file=entityEmb_file,
                          contextEmb_file=contextEmb_file,
                          epochs=epochs,
                          history_size=history_size,
                          batch_size=batch_size)
print(hparams)

W0727 10:09:31.440346 139756242384704 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



kg_file=None,user_clicks=None,FEATURE_COUNT=None,FIELD_COUNT=None,data_format=dkn,PAIR_NUM=None,DNN_FIELD_NUM=None,n_user=None,n_item=None,n_user_attr=None,n_item_attr=None,iterator_type=None,SUMMARIES_DIR=None,MODEL_DIR=None,wordEmb_file=/tmp/tmp1rtcnx4s/mind-demo-dkn/word_embeddings_100.npy,entityEmb_file=/tmp/tmp1rtcnx4s/mind-demo-dkn/TransE_entity2vec_100.npy,contextEmb_file=/tmp/tmp1rtcnx4s/mind-demo-dkn/TransE_context2vec_100.npy,news_feature_file=/tmp/tmp1rtcnx4s/mind-demo-dkn/doc_feature.txt,user_history_file=/tmp/tmp1rtcnx4s/mind-demo-dkn/user_history.txt,use_entity=True,use_context=True,doc_size=10,history_size=50,word_size=12600,entity_size=3987,entity_dim=100,entity_embedding_method=None,transform=True,train_ratio=None,dim=100,layer_sizes=[300],cross_layer_sizes=None,cross_layers=None,activation=['sigmoid'],cross_activation=identity,user_dropout=False,dropout=[0.0],attention_layer_sizes=100,attention_activation=relu,attention_dropout=0.0,model_type=dkn,method=classification

## Train the DKN model

In [5]:
model = DKN(hparams, DKNTextIterator)

W0727 10:09:32.228476 139756242384704 deprecation.py:323] From /home/miguel/anaconda/envs/reco_gpu/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:2825: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0727 10:09:32.514580 139756242384704 deprecation.py:323] From ../../reco_utils/recommender/deeprec/models/dkn.py:308: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W0727 10:09:32.516178 139756242384704 deprecation.py:323] From /home/miguel/anaconda/envs/

In [6]:
print(model.run_eval(valid_file))

{'auc': 0.511, 'group_auc': 0.4929, 'mean_mrr': 0.1409, 'ndcg@5': 0.1357, 'ndcg@10': 0.1988}


In [7]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:0.6707635026867107
eval info: auc:0.5601, group_auc:0.5114, mean_mrr:0.172, ndcg@10:0.2394, ndcg@5:0.1708
at epoch 1 , train time: 76.3 eval time: 11.4
at epoch 2
train info: logloss loss:0.6216596258898913
eval info: auc:0.5653, group_auc:0.5307, mean_mrr:0.1798, ndcg@10:0.2427, ndcg@5:0.1852
at epoch 2 , train time: 74.7 eval time: 11.6
at epoch 3
train info: logloss loss:0.5895718016614349
eval info: auc:0.5854, group_auc:0.5445, mean_mrr:0.1818, ndcg@10:0.2448, ndcg@5:0.1909
at epoch 3 , train time: 75.0 eval time: 11.5
at epoch 4
train info: logloss loss:0.5658008101380477
eval info: auc:0.5891, group_auc:0.5446, mean_mrr:0.1822, ndcg@10:0.2504, ndcg@5:0.1813
at epoch 4 , train time: 74.8 eval time: 11.5
at epoch 5
train info: logloss loss:0.547249739958068
eval info: auc:0.5764, group_auc:0.5449, mean_mrr:0.19, ndcg@10:0.256, ndcg@5:0.1897
at epoch 5 , train time: 74.5 eval time: 11.6
at epoch 6
train info: logloss loss:0.5314540236683215
eval 

## Evaluate the DKN model

Now we can check the performance on the test set:

In [10]:
res = model.run_eval(test_file)
print(res)

{'auc': 0.5988, 'group_auc': 0.5637, 'mean_mrr': 0.1893, 'ndcg@5': 0.187, 'ndcg@10': 0.2494}


In [ ]:
sb.glue("res", res)

## References

\[1\] Wang, Hongwei, et al. "DKN: Deep Knowledge-Aware Network for News Recommendation." Proceedings of the 2018 World Wide Web Conference on World Wide Web. International World Wide Web Conferences Steering Committee, 2018.<br>
\[2\] Knowledge Graph Embeddings including TransE, TransH, TransR and PTransE. https://github.com/thunlp/KB2E <br>
\[3\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[4\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/